<a href="https://colab.research.google.com/github/Rey1380/WavLM/blob/main/Copy_of_wavLM_base_AGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WavLM_base

In [29]:
!pip install time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [30]:
import time
start = time.time()

In [31]:
#!pip install  transformers 
#!pip install --quiet datasets
!pip install pycm==3.5
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
import torch
import librosa
import IPython.display as ipd
import numpy as np

In [36]:
noise,sr=librosa.load("/content/drive/MyDrive/intern/noise-sound-bible-0061.wav")

In [37]:
len(noise)

429182

In [39]:
import os
spk2spch = {}
PATH = "/content/drive/MyDrive/intern/farsdot_enrollment2/"
for fname in next(os.walk(PATH))[2]:
    spk_id, _ = fname.split("F")
    speech_array, sample_rate = librosa.load(PATH + fname)
    speech_array=mix_audio(speech_array,noise,15)
    speech_array = speech_array.squeeze().astype(np.float)
    speech_array = librosa.resample(np.array(speech_array), sample_rate, 16000)
    
    spk2spch[f"spk-{spk_id}"] = speech_array
print(spk2spch)

0.14024490191727887 0.9903084168330514 0.138885706786607
0.12705074303746358 0.9920254685340317 0.12603757288933667


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


0.2837431816601341 0.9620232450814031 0.2729675363904043
0.15553687952561698 0.988119271613991 0.15368898810596573
0.12709074712763552 0.9920205058664985 0.1260766272565082
0.09522905072303592 0.9954963223490693 0.09480016977557523
0.1623373762447781 0.9870781360334604 0.16023967475225812
0.2010681464630253 0.9803787763847054 0.19712294339936146
0.1116215864976244 0.9938279263526153 0.11093264984512313
0.14464134741206688 0.9897007651164407 0.1431516522011955
0.22019070544876612 0.9766053684499817 0.21503942502405363
0.1648053731369935 0.9866901193752238 0.1626118332942184
0.31326249873361595 0.9542725929248632 0.29893781693264937
0.2056336180959788 0.9795051362153342 0.20141918510355372
0.2090370382136556 0.9788426631026206 0.2046143711721389
0.2935478621770473 0.959513366036307 0.2816630973302606
0.26896199015575945 0.9656808854016118 0.2597314527929934
0.25621107003395427 0.9687102723300264 0.2481942954265593
0.20565572790228676 0.9795008633450059 0.2014399630321355
0.24340207795324

In [14]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.4 MB/s 
     |████████████████████████████████| 115 kB 66.0 MB/s 
     |████████████████████████████████| 212 kB 58.0 MB/s 
     |████████████████████████████████| 141 kB 61.4 MB/s 
     |████████████████████████████████| 127 kB 53.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [40]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
from datasets import load_dataset
import torch
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("microsoft/wavlm-base-plus-sv")
model = WavLMForXVector.from_pretrained("microsoft/wavlm-base-plus-sv")
threshold = 0.9
similarities = []

In [27]:
len(speech_array)

96000

In [ ]:
from os.path import exists

with open('/content/drive/MyDrive/intern/farsdot/trials') as f:
    for line in f:
        line = line.strip()
        spk_id, utt_id, target_nontarget = line.split()
        path = "/content/drive/MyDrive/intern/wavlm large/farsdot/"  + utt_id.split("-")[1] + ".wav.wav"
        if exists(path) is False:
            path = "/content/drive/MyDrive/intern/wavlm large/farsdot/"  + utt_id.split("-")[1] + ".WAV.WAV"
        
        speech_array2, sample_rate =librosa.load(path)
        speech_array2=mix_audio(speech_array2,noise,15)
        print(path, spk_id, target_nontarget)
        speech_array2 = speech_array2.squeeze().astype(float)
        speech_array2 = librosa.resample(np.array(speech_array2), sample_rate, 16000)
        speech3 = [spk2spch[spk_id], speech_array2]
        
        inputs= feature_extractor(
            speech3,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True)

        with torch.no_grad():
            embeddings = model(**inputs).embeddings

        embeddings = torch.nn.functional.normalize(embeddings, dim=-1).cpu()
        cosine_sim = torch.nn.CosineSimilarity(dim=-1)
        similarity = cosine_sim(embeddings[0], embeddings[1])
        similarities.append(similarity.item())
        print(similarity.item())

0.3518181510591148 0.9433223438063495 0.33187792285070045
/content/drive/MyDrive/intern/wavlm large/farsdot/001F02.WAV.WAV spk-001 target


/usr/local/lib/python3.7/dist-packages/transformers/models/wavlm/modeling_wavlm.py:1755: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


0.9720351696014404
0.34904543900153306 0.9441390245942421 0.32954742031797646
/content/drive/MyDrive/intern/wavlm large/farsdot/001F03.WAV.WAV spk-001 target
0.9906752705574036
0.3338226609204826 0.9485439614617145 0.3166454692152052
/content/drive/MyDrive/intern/wavlm large/farsdot/001F04.WAV.WAV spk-001 target
0.9724102020263672
0.345803490041569 0.945088380538841 0.32681486038806573
/content/drive/MyDrive/intern/wavlm large/farsdot/001F05.WAV.WAV spk-001 target
0.9788296222686768
0.31737214829136245 0.9531485003589271 0.3025027871996031
/content/drive/MyDrive/intern/wavlm large/farsdot/001F06.WAV.WAV spk-001 target
0.9721957445144653
0.29472532816265684 0.9592075622489541 0.2827027635599251
/content/drive/MyDrive/intern/wavlm large/farsdot/001F07.WAV.WAV spk-001 target
0.987989068031311
0.3598440521275889 0.9409341674245917 0.3385895635913642
/content/drive/MyDrive/intern/wavlm large/farsdot/001F08.WAV.WAV spk-001 target
0.9774734973907471
0.282431863622122 0.9623539261551919 0.2717

In [20]:
len(speech_array2)


96000

In [21]:
len(noise)

2

In [ ]:
np.save("similarities.npy", np.array(similarities))

In [ ]:
actual_vector = []
with open('/content/drive/MyDrive/intern/wavlm large/farsdot/trials') as f:
    for line in f:
        line = line.strip()
        spk_id, utt_id, target_nontarget = line.split()
        if target_nontarget == "nontarget":
            actual_vector.append(0)
        else:
            actual_vector.append(1)
actual_vector = np.array(actual_vector)

ROC:

In [ ]:
threshold = 0.957

In [ ]:
similarities = np.array(similarities)
predict_vector = (similarities > threshold) * 1
print(predict_vector)
print(actual_vector)

In [ ]:
np.save("actual_vector.npy", np.array(actual_vector))

In [ ]:
actual_vector = np.load("actual_vector.npy")
predict_vector = np.load("predict_vector.npy")
similarities = np.load("similarities.npy")

In [ ]:
sum(predict_vector != actual_vector) / len(actual_vector)

In [ ]:
from pycm import ConfusionMatrix
cm = ConfusionMatrix(actual_vector, predict_vector)
cm

In [ ]:
cm.plot()

In [ ]:
print(cm)

In [ ]:
print(cm.overall_stat["F1 Macro"], cm.overall_stat["F1 Micro"])

In [ ]:
for threshold in np.arange(0.94, 0.98, 0.001):
    predict_vector = (similarities > threshold) * 1
    cm = ConfusionMatrix(actual_vector, predict_vector)
    print(threshold, cm.overall_stat["F1 Macro"], cm.overall_stat["F1 Micro"])

In [ ]:
print(speech3)

In [ ]:
inputs= feature_extractor(
  speech3, sampling_rate=16000, return_tensors="pt", padding=True
    )

with torch.no_grad():
    embeddings = model(**inputs).embeddings

embeddings = torch.nn.functional.normalize(embeddings, dim=-1).cpu()

# the resulting embeddings can be used for cosine similarity-based retrieval
cosine_sim = torch.nn.CosineSimilarity(dim=-1)
similarity = cosine_sim(embeddings[0], embeddings[1])
threshold = 0.9  # the optimal threshold is dataset-dependent
if similarity < threshold:
    print("Speakers are not the same!")
round(similarity.item(), 2)

In [ ]:
from pycm import *
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import os

In [1]:
def mix_audio(signal, noise, snr):
    # if the audio is longer than the noise
    # play the noise in repeat for the duration of the audio
    noise = noise[np.arange(len(signal)) % len(noise)]
    
    # if the audio is shorter than the noise
    # this is important if loading resulted in 
    # uint8 or uint16 types, because it would cause overflow
    # when squaring and calculating mean
    noise = noise.astype(np.float32)
    signal = signal.astype(np.float32)
    
    # get the initial energy for reference
    signal_energy = np.mean(signal**2)
    noise_energy = np.mean(noise**2)
    # calculates the gain to be applied to the noise 
    # to achieve the given SNR
    g = np.sqrt(10.0 ** (-snr/10) * signal_energy / noise_energy)
    
    # Assumes signal and noise to be decorrelated
    # and calculate (a, b) such that energy of 
    # a*signal + b*noise matches the energy of the input signal
    a = np.sqrt(1 / (1 + g**2))
    b = np.sqrt(g**2 / (1 + g**2))
    print(g, a, b)
    # mix the signals
    return a * signal + b * noise

In [ ]:
end = time.time()
print("Execution time of the program is(RTF)- ", end-start)